In [28]:
from IPython.display import clear_output
from tinydb import TinyDB, Query
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup

import re

In [29]:
db = TinyDB('data/custom_data/db.json')

In [30]:
players = db.table('players')
tournaments = db.table('tournaments')
matches = db.table('matches')

In [33]:
query = Query()
START_INDEX = 25724
END_INDEX = 52928
for player_id in range(START_INDEX,END_INDEX+1):
    clear_output(wait=True)
    print("Player ID",player_id)
    url_tab = "https://www.ultimatetennisstatistics.com/playerProfileTab?playerId="+str(player_id)
    url_page = "https://www.ultimatetennisstatistics.com/playerProfile?playerId="+str(player_id)
    player_name = None
    try:
        f = urlopen(url_page)
        re_obj = re.compile(r'.*(<head.*<title.*?>(.*)</title>.*</head>)',re.DOTALL)
        Found = False
        data = ''
        while True:
            b_data = f.read(4096)
            if not b_data: break

            data += b_data.decode(errors='ignore')
            match = re_obj.match(data)
            if match:
                Found = True
                title = match.groups()[1]
                player_name = title.replace("Ultimate Tennis Statistics - ","").strip()
                break

        f.close()
    except HTTPError as e:
        if e.getcode() == 404:
            continue
        raise

    page = requests.get(url_tab)
    soup = BeautifulSoup(page.text, 'html.parser')
    info = {"id":player_id,"name":player_name,"aliases":[player_name]}
    for table in soup.find_all("table",class_=["table","table-condensed","text-nowrap"]):
        for row in table.find_all("tr"):
            if row.find("th") is None or row.find("td") is None:
                heads = row.find_all("th")
                if len(heads) == 3:
                    if not "surfaces" in info:
                        info["surfaces"] = {}
                    surface = heads[0].text.strip()
                    info["surfaces"][surface] = {"raw_matches":heads[1].text.strip()}
                    if surface != "Overall":
                        info["surfaces"][surface]["tournament_wins"] = int(heads[2].text.strip())
                continue
            info[row.find("th").text.strip()]=row.find("td").text.strip()
    players.remove(query.id==player_id)        
    players.insert(info)

Player ID 31957


ConnectionError: HTTPSConnectionPool(host='www.ultimatetennisstatistics.com', port=443): Max retries exceeded with url: /playerProfileTab?playerId=31957 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001FFD195A898>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [23]:
# players.purge()

In [ ]:
# player_id=3819
# url = "https://www.ultimatetennisstatistics.com/playerProfileTab?playerId="+str(player_id)


# page = requests.get(url)
# soup = BeautifulSoup(page.text, 'html.parser')
# info = {"id":player_id}
# for table in soup.find_all("table",class_=["table","table-condensed","text-nowrap"]):
#     for row in table.find_all("tr"):
#         if row.find("th") is None or row.find("td") is None:
#             heads = row.find_all("th")
#             if len(heads) == 3:
#                 if not "surfaces" in info:
#                     info["surfaces"] = {}
#                 surface = heads[0].text.strip()
#                 info["surfaces"][surface] = {"raw_matches":heads[1].text.strip()}
#                 if surface != "Overall":
#                     info["surfaces"][surface]["tournament_wins"] = int(heads[2].text.strip())
                    
#             continue
#         info[row.find("th").text.strip()]=row.find("td").text.strip()
# players.remove(query.id==player_id)        
# players.insert(info)
# info